# Student District Mapper

## Instructions

This guide will walk you through the process of using the script on mybinder.org. This script reads an Excel file containing student data, geocodes the students' addresses using the Google Maps API, and associates the students with their corresponding LA Council Districts using a spatial join. The resulting data is saved as a CSV file.

## Step 1: Prepare the input files

1. Ensure that the Excel file containing the student data is named `stem-prep-students.xlsx` and has the following columns: `Student_Number`, `street`, `city`, `state`, and `zip`. 

2. Obtain the shapefile of the LA Council Districts, which should include a `.shp` file and accompanying files with the same name and different extensions (such as `.dbf`, `.shx`, and `.prj`). Name the main shapefile `la-council-districts.shp`.

## Step 2: Upload the input files to the mybinder.org environment

1. Navigate to [mybinder.org](https://mybinder.org/) in your web browser.

2. Enter the GitHub repository URL containing the Jupyter Notebook with the script and click "launch" to create a new Jupyter Notebook instance.

3. Once the environment is ready, click the "Upload" button in the top-right corner of the Jupyter Notebook dashboard.

4. Select the `stem-prep-students.xlsx` file and all of the LA Council Districts shapefile components (`.shp`, `.dbf`, `.shx`, `.prj`, and any other accompanying files) and click "Open" to upload them.

## Step 3: Run the script in the Jupyter Notebook

1. Open the Jupyter Notebook containing the script.

2. Ensure that the Google Maps API key is properly set in the `geocode_address` function, in the field called [YOUR-GOOGLE-MAPS-API-KEY]. The key should be a string enclosed in single quotes and assigned to the `key` parameter when creating the `googlemaps.Client` object.

3. In the toolbar at the top of the Jupyter Notebook, click "Kernel" and then "Restart & Run All" to execute the script.

4. The script will print progress messages as it geocodes the addresses and performs the spatial join. When it finishes, the output DataFrame containing the student data and associated council district information will be displayed.

## Step 4: Download the output file

1. Once the script has finished running, the output CSV file named `output_stem-prep-cds.csv` will be generated in the mybinder.org environment.

2. In the Jupyter Notebook dashboard, locate the `output_stem-prep-cds.csv` file.

3. Click the checkbox next to the `output_stem-prep-cds.csv` file to select it.

4. Click the "Download" button in the top-right corner to download the file to your computer.


## Code

In [14]:
# Install dependencies
!pip install -U googlemaps
!pip install pandas
!pip install geopandas shapely
!pip install geopy
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import googlemaps
from concurrent.futures import ThreadPoolExecutor, as_completed

In [13]:
# Read student data as a DataFrame
data = pd.read_excel('stem-prep-students.xlsx')
students_df = pd.DataFrame(data)

# Read LA Council Districts map shapefile
la_council_districts = gpd.read_file('la-council-districts.shp')

# Function to geocode an address using Google Maps API
def geocode_address(address, retries=3):
    gmaps = googlemaps.Client(key='[YOUR-GOOGLE-MAPS-API-KEY]')
    
    for i in range(retries):
        try:
            geocode_result = gmaps.geocode(address)
            if geocode_result and len(geocode_result) > 0:
                location = geocode_result[0]['geometry']['location']
                return location['lat'], location['lng']
            else:
                print(f"Failed to geocode '{address}'")
                return None, None
        except Exception as e:
            print(f"Error geocoding '{address}': {e}")
            if i < retries - 1:
                print("Retrying...")

# Concatenate address components to form full address
students_df['full_address'] = students_df['street'] + ', ' + students_df['city'] + ', ' + students_df['state'] + ', ' + students_df['zip'].apply(lambda x: f'{int(x):05d}')
print("Geocoding addresses...")

# Function to update geocoded results for each address
def update_geocoded_result(idx, address):
    lat, lng = geocode_address(address)
    return idx, lat, lng

# Geocode addresses concurrently using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = {executor.submit(update_geocoded_result, idx, address): idx for idx, address in enumerate(students_df['full_address'])}

    for future in as_completed(futures):
        idx, lat, lng = future.result()
        students_df.at[idx, 'latitude'] = lat
        students_df.at[idx, 'longitude'] = lng
        print(f"Successfully geocoded address {idx + 1}/{len(students_df)}")

# Create point geometry and convert to GeoDataFrame
students_df['geometry'] = students_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
students_gdf = gpd.GeoDataFrame(students_df, geometry='geometry', crs='EPSG:4326')

# Reproject LA Council Districts to match the CRS of students_gdf
la_council_districts = la_council_districts.to_crs(students_gdf.crs)

# Perform spatial join to associate students with their council districts
students_with_districts = gpd.sjoin(students_gdf, la_council_districts, how='left', predicate='within')

# Merge council district information back to the original DataFrame
students_df['district'] = students_with_districts['district'].astype(int)
students_df = students_df.drop(['full_address','geometry'], axis=1).rename(columns={'Student_Number':'student_number'})

# Save the resulting DataFrame to a CSV file
students_df.to_csv('output_stem-prep-cds.csv', index=False)
students_df

Geocoding addresses...
Geocoded address 8/10
Geocoded address 7/10
Geocoded address 1/10
Geocoded address 6/10
Geocoded address 3/10
Geocoded address 2/10
Geocoded address 4/10
Geocoded address 5/10
Geocoded address 10/10
Geocoded address 9/10


,student_number,grade_level,street,city,state,zip,latitude,longitude,district
0,19560,5,1662 1/2 S Hobart Blvd,Los Angeles,CA,90006,34.042783,-118.304988,10
1,19531,5,"1723 South Oxford Avenue, Apt # 2",Los Angeles,CA,90006,34.041925,-118.307952,10
2,19632,5,"2646 S. Manhattan Pl., Apt. #202",Los Angeles,CA,90018,34.031288,-118.309661,10
3,19585,5,1116 Fedora St. Apt. # 1,Los Angeles,CA,90006,34.050355,-118.297658,1
4,19554,5,2371 West 18th St. Apt. # 3,Los Angeles,CA,90019,34.041943,-118.311240,10
5,19530,5,3917 Denker Ave.,Los Angeles,CA,90062,34.013899,-118.304826,8
6,19537,5,"2199 W 26th Pl., Apt #25",Los Angeles,CA,90018,34.032149,-118.314901,10
7,19747,5,"2016 W. Adams Blvd, # 108",Los Angeles,CA,90018,34.032565,-118.309620,10
8,19550,5,"2651 S. Manhattan Pl., Apt. # 201",Los Angeles,CA,90018,34.031127,-118.310327,10
9,19558,5,1973 W. 22nd St. Apt. # 6,Los Angeles,CA,90018,34.036495,-118.305879,8
